<a href="https://colab.research.google.com/github/softcomput/cp-colab/blob/main/cpp_to_work_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

1. Install Google OR-Tools.
2. Create small data model or large data model.
3. Run car pooling problem to work.

In [ ]:
!pip install ortools

In [ ]:
import logging
import re

from ortools.linear_solver import pywraplp
import numpy as np


class CPPToWork:
    """CPPToWork

    Attributes:
        data (dict):
            Vs (list): servers
            Vc (list): clients
            Q (list): capacities
            T (list): maximum ride times
            Ls (list): latest arrival time at workspace of servers
            Lc (list): latest arrival time at workspace of clients
            Ss (list): earliest departure time　at home of servers
            Sc (list): earliest departure time　at home of clients
            distance (2-d list): distance for each server and client
            duration (2-d list): duration for each server and client
            risk (2-d list): risk for each server and client
    """

    def __init__(self, data):
        self.d = self.__format_data(data)
        self.M = 10000

    def __format_data(self, data):
        """ __format_data

        Format data model.
        """
        distance = []
        for vs_d in data['distance'][0:len(data['Vs'])]:
            distance.append(vs_d)
            distance.extend(data['distance'][len(data['Vs']):])
        data['distance'] = distance

        duration = []
        for vs_d in data['duration'][0:len(data['Vs'])]:
            duration.append(vs_d)
            duration.extend(data['duration'][len(data['Vs']):])
        data['duration'] = duration

        distance_vc = []
        for vc in range(len(data['Vc'])):
            distance_vc.append(data['distance'][vc + 1][0])
        data['distance_vc'] = distance_vc

        l_vc = []
        for vs in range(len(data['Vs'])):
            l = []
            l.append(data['Ls'][vs])
            l.extend(data['Lc'])
            l_vc.append(l)
        data['L'] = l_vc

        s_vc = []
        for vs in range(len(data['Vs'])):
            s = []
            s.append(data['Ss'][vs])
            s.extend(data['Sc'])
            s_vc.append(s)
        data['S'] = s_vc

        return data

    def __create_variables(self, solver):
        """ __create_variables

        Create variables.

        Returns:
            x (list): x^k_ij, k in Vs
            y (list): y_i, i in Vc
            u (list): u^k_i, Si <= u^k_i <= Li　- t_i0
            v (list): v^k, Sk + t_k0 <= v^k <= Lk　, k in Vs
        """
        x = []
        u = []
        t_k0 = []
        for idx, k in enumerate(self.d['Vs']):
            ai = [k]
            ai.extend(self.d['Vc'])
            aj = [0]
            aj.extend(self.d['Vc'])
            for i in ai:
                x.append([
                    solver.IntVar(0, 1, 'x{}({},{})'.format(k, i, j))
                    for j in aj])
            
            t_k0_vs = self.d['duration'][(1 + len(self.d['Vc'])) * idx][0]
            t_i0 = [t_k0_vs]
            t_k0.append(t_k0_vs)
            for idx_i in range(len(ai[1:])):
                t_i0.append(self.d['duration'][
                    1 + (1 + len(self.d['Vc'])) * idx + idx_i][0])

            u.append([solver.IntVar(
                self.d['S'][idx][idx_i],
                self.d['L'][idx][idx_i] - t_i0[idx_i],
                'u{}({})'.format(k, i)) for idx_i, i in enumerate(ai)])
            
            self.d['duration'][(1 + len(self.d['Vc'])) * idx][0]

        y = [solver.IntVar(0, 1, 'y({})'.format(i)) for i in self.d['Vc']]

        v = [solver.IntVar(
            data['Ss'][idx] + t_k0[idx],
            data['Ls'][idx],
            'v{}'.format(k)) for idx, k in enumerate(data['Vs'])]

        return x, y, u, v

    def __create_constraints(self, solver, x, y, u, v):
        """ __create_constraints

        Create constraints.
        """
        n = len(self.d['Vc']) + 1

        for vs in range(len(self.d['Vs'])):
            k = vs * n
            # Each employee leaves its house.
            solver.Add(solver.Sum(x[k]) == 1)

            # Each employee arrives at the workspace.
            solver.Add(solver.Sum([x[j][0] for j in range(k, k + n)]) == 1)

            # Continuity.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(
                    solver.Sum([x[j][i] for j in range(k, k + n)]) -
                    solver.Sum(x[k + i]) == 0)

            # Capacity.
            solver.Add(
                solver.Sum([x_k for i in range(n) for x_k in x[k + i]]) <=
                self.d['Q'][vs])

            # Maximum ride time.
            solver.Add(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'][k:k + n], x[k:k + n])]) <=
                self.d['T'][vs])

            # Own arc is not passed.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(x[k + i][i] == 0)

            # Departure time condition with subtour elimination constraints by MTZ.
            for j in range(1, len(u[vs])):
                solver.Add(
                    u[vs][0] +
                    self.d['duration'][k][j] -
                    self.M * (1 - x[k][j]) <=
                    u[vs][j])
            for i in range(1, len(u[vs])):
                for j in range(1, len(u[vs])):
                    if str(u[vs][i]) != str(u[vs][j]):
                        solver.Add(
                            u[vs][i] +
                            self.d['duration'][k + i][j] -
                            self.M * (1 - x[k + i][j]) <=
                            u[vs][j])
            solver.Add(u[vs][0] == 1)

            # Arrival time at workspace of server.
            for i in range(len(u[vs])):
                solver.Add(
                    v[vs] >=
                    u[vs][i] +
                    self.d['duration'][k + i][0] -
                    self.M * (1 - x[k + i][0]))
                
                solver.Add(
                    v[vs] <=
                    self.d['L'][vs][i] +
                    self.M * (1 - solver.Sum(x[k + i])))

        # Each client is either picked up by a server or is left unserviced.
        for vc in range(len(self.d['Vc'])):
            solver.Add(
                solver.Sum([x_k for vs in range(len(self.d['Vs']))
                           for x_k in x[vs * n + vc + 1]]) + y[vc] == 1)

    def __create_objective(self, solver, x, y):
        """ __create_objective

        Create objective.
        """
        if self.d.get('risk'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['risk']) *
                    np.array(self.d['distance']), x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))
        else:
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['distance'], x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))

    def solve(self):
        """ solve

        Solve CPP.
        """
        solver = pywraplp.Solver(
            'cp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        x, y, u, v = self.__create_variables(solver)
        self.__create_constraints(solver, x, y, u, v)
        self.__create_objective(solver, x, y)

        status = solver.Solve()

        paths = {}
        clients_not_pickedup = []
        if status == pywraplp.Solver.OPTIMAL:
            n = len(self.d['Vc']) + 1
            root = []
            for i in range(len(self.d['Vs']) * n):
                for j in range(n):
                    if x[i][j].solution_value():
                        root.append(x[i][j].name())
                        r = re.findall(r'\w+', x[i][j].name())
                        r[0] = r[0].lstrip('x')
                        if not r[0] in paths:
                            paths[r[0]] = []
                        paths[r[0]].append((r[1], r[2]))
            paths = self.__format_paths(paths)
            for i in range(len(self.d['Vc'])):
                if y[i].solution_value():
                    r = re.findall(r'\w+', y[i].name())
                    clients_not_pickedup.append(r[1])
            arrival_times = []
            for i in range(len(self.d['Vs'])):
                arrival_times.append([
                    (u[i][j].name(), u[i][j].solution_value())
                    for j in range(len(u[i]))])
            logging.info(
                'Root %s, clients not picked up %s, \
                and arrival_times %s solved in %d variables, \
                %d constraints, %f objective value, %f milliseconds, \
                %d iterations, and %d branch-and-bound nodes.' % (
                    root, clients_not_pickedup, arrival_times,
                    solver.NumVariables(),
                    solver.NumConstraints(),
                    solver.Objective().Value(),
                    solver.wall_time(), solver.iterations(), solver.nodes()))
        else:
            logging.info('The problem does not have an optimal solution.')
        m, tdr, dti = self.__evaluate(x, y)
        er = self.__evaluate_accident_evasion_rate(x, y)
        logging.info('evasion_rate: %s', er)
        print('evasion_rate: %s', er)
        return {
            'feasible_paths': paths,
            '%m': m,
            '%tdr': tdr,
            '%dti': dti,
            'clients_not_pickedup': clients_not_pickedup
        }

    def __format_paths(self, paths):
        for i in paths:
            clients = []
            next = paths[i][0][1]
            for _ in paths[i]:
                if not next == '0':
                    for j in paths[i]:
                        if next == j[0]:
                            clients.append(j[0])
                            next = j[1]
                            break
            paths[i] = clients
        return paths

    def __evaluate(self, x, y):
        total_distance = 0
        total_duration = 0
        original_distance = 0
        original_duration_vs = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_distance += self.d['distance'][i][j]
                    total_duration += self.d['duration'][i][j]
        for i in range(len(self.d['Vs'])):
            original_distance += self.d['distance'][i * n][0]
            original_duration_vs += self.d['duration'][i * n][0]
        not_pickup_clients = 0
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                not_pickup_clients += 1
                total_distance += self.d['distance'][i + 1][0]
            original_distance += self.d['distance'][i + 1][0]

        m = (len(self.d['Vc']) - not_pickup_clients) / len(self.d['Vc'])
        tdr = 1 - total_distance / original_distance
        dti = total_duration / original_duration_vs
        return m, tdr, dti

    def __evaluate_accident_evasion_rate(self, x, y):
        if not self.d.get('accidentCount'):
            return 0
        total_accident_count = 0
        original_accident_count = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_accident_count += self.d['accidentCount'][i][j]
        for i in range(len(self.d['Vs'])):
            original_accident_count += self.d['accidentCount'][i * n][0]
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                total_accident_count += self.d['accidentCount'][i + 1][0]
            original_accident_count += self.d['accidentCount'][i + 1][0]

        evasion_rate = 1 - total_accident_count / original_accident_count
        return evasion_rate



data = create_data_model()
output = CPPToWork(data).solve()
print(output)

evasion_rate: %s 0
{'feasible_paths': {'1': ['6', '7', '3', '4', '8'], '2': ['5']}, '%m': 0.8571428571428571, '%tdr': 0.19867549668874174, '%rti': 1.05, 'clients_not_pickedup': ['9']}


In [ ]:
# def create_data_model():
#     data = {}
#     data['Vs'] = [1,3,4,6,9]
#     data['Vc'] = [2,5,7,8,10]
#     data['Q'] = [2,2,2,2,2]
#     data['T'] = [3600,5400,7200,7200,3600]
#     data['Ls'] = [7200,7200,7200,7200,7200]
#     data['Lc'] = [7200,7200,7200,7200,7200]
#     data['Ss'] = [0,0,0,0,0]
#     data['Sc'] = [0,0,0,0,0]
#     data['distance'] = [[6403,5613,5816,7897,12460,9567],[25968,25256,19491,27462,32025,29132],[8936,11604,12648,4695,12829,11164],[17263,13158,10410,20811,20755,17400],[7230,9898,10942,2989,11122,9457],[4833,0,3687,7743,10891,7998],[9563,3687,0,13111,10236,7343],[4562,7743,13111,0,8454,6789],[6174,10891,10236,8454,0,3590],[3322,7998,7343,6789,3590,0]]
#     data['duration'] = [[776,975,1010,1055,1578,1190],[2083,2302,2208,2362,2885,2497],[1141,1392,1458,774,1603,1491],[1493,1844,1330,1849,1833,1582],[865,1105,1171,498,1327,1215],[657,0,648,904,1459,1071],[942,648,0,1298,1094,706],[459,904,1298,0,921,809],[851,1459,1094,921,0,526],[513,1071,706,809,526,0]]
#     # data['risk'] = [[1,1,1,1,1,1],[1,0,1,1,1,1],[1,1,0,1,1,1],[1,1,1,0,1,1],[1,1,1,1,0,1],[1,1,1,1,1,0],[1,1,1,1,1,1],[1,0,1,1,1,1],[1,1,0,1,1,1],[1,1,1,0,1,1],[1,1,1,1,0,1],[1,1,1,1,1,0],[1,1,1,1,1,1],[1,0,1,1,1,1],[1,1,0,1,1,1],[1,1,1,0,1,1],[1,1,1,1,0,1],[1,1,1,1,1,0],[1,1,1,1,1,1],[1,0,1,1,1,1],[1,1,0,1,1,1],[1,1,1,0,1,1],[1,1,1,1,0,1],[1,1,1,1,1,0]]
#     # data['accidentCount'] = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]
#     return data

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['Vs'] = ['1', '2']
    data['Vc'] = ['3', '4', '5', '6', '7', '8', '9']
    data['Q'] = [7, 5]
    data['T'] = [12, 12]
    data['Ls'] = [12, 12]
    data['Lc'] = [12, 12, 12, 12, 12, 12, 100]
    data['Ss'] = [0, 0]
    data['Sc'] = [0, 0, 0, 0, 0, 0, 0]
    data['distance'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    data['duration'] = [
                        [10, 5, 7, 10, 1, 3, 10, 100],
                        [10, 7, 9, 4, 11, 11, 12, 100],
                        [5, 0, 2, 5, 4, 2, 5, 100],
                        [3, 2, 0, 5, 6, 4, 3, 100],
                        [6, 5, 5, 0, 9, 7, 8, 100],
                        [9, 4, 6, 9, 0, 2, 9, 100],
                        [7, 2, 4, 7, 2, 0, 7, 100],
                        [1, 5, 3, 8, 9, 7, 0, 100],
                        [100, 100, 100, 100, 100, 100, 100, 0],
                        ]
    return data